# Recognition of gestures and actions


## Detecting body points

### Importing the libraries

In [1]:
import cv2
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow
import numpy as np

### Loading the image

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#image = cv2.imread('/content/drive/MyDrive/vision/megan.jpg')
image = cv2.imread('images/megan.jpg')

In [6]:
cv2.imshow('megan.jpg', image)

In [7]:
image.shape, image.shape[0] * image.shape[1] * 3

((337, 600, 3), 606600)

In [8]:
type(image)

numpy.ndarray

In [9]:
# Mean subtraction: https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/
image_blob = cv2.dnn.blobFromImage(image = image, scalefactor = 1.0 / 255,
                                   size = (image.shape[1], image.shape[0]))

In [10]:
type(image_blob), image_blob.shape # batch

(numpy.ndarray, (1, 3, 337, 600))

### Loading the pre-trained neural network

- Caffe Deep Learning framework: https://caffe.berkeleyvision.org/

In [26]:
# network = cv2.dnn.readNetFromCaffe('/content/drive/MyDrive/Weights/pose_deploy_linevec_faster_4_stages.prototxt',
#                                   '/content/drive/MyDrive/Weights/pose_iter_160000.caffemodel')

network = cv2.dnn.readNetFromCaffe('openpose-master/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt', 'pose_iter_160000.caffemodel')

error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/dnn/src/caffe/caffe_io.cpp:1138: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "pose_iter_160000.caffemodel" in function 'ReadProtoFromBinaryFile'


In [ ]:
network.getLayerNames()

In [ ]:
len(network.getLayerNames())

### Predicting body points

In [14]:
network.setInput(image_blob)
output = network.forward()

In [ ]:
# 44 -> related to the points that were detected
# (43, 75) -> information about the location
output.shape

In [16]:
position_width = output.shape[3]
position_heigth = output.shape[2]

In [ ]:
position_width

In [ ]:
(image.shape[1] * 28) / 75

In [19]:
# minMaxLoc: https://docs.opencv.org/master/d2/de8/group__core__array.html#gab473bf2eb6d14ff97e89b355dac20707
num_points = 15
points = []
threshold = 0.1
for i in range(num_points):
  #print(i)
  confidence_map = output[0, i, :, :]
  #print(confidence_map) # candidate points
  #print(len(confidence_map))
  _, confidence, _, point = cv2.minMaxLoc(confidence_map)
  #print(confidence)
  #print(point)

  x = int((image.shape[1] * point[0]) / position_width)
  y = int((image.shape[0] * point[1]) / position_heigth)
  #print(x,y)
  if confidence > threshold:
    cv2.circle(image, (x, y), 5, (0,255,0), thickness = -1)
    cv2.putText(image, '{}'.format(i), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255))
    points.append((x,y))
  else:
    points.append(None)

In [ ]:
points

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB));

In [22]:
point_connections = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7],[1,14],
                     [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]

In [ ]:
point_connections

In [28]:
for connection in point_connections:
  #print(connection)
  partA = connection[0]
  partB = connection[1]
  #print(partA, partB)
  if points[partA] and points[partB]:
    cv2.line(image, points[partA], points[partB], (255,0,0))

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB));

## Detecting movements (arms above the head)

### Arms above the head in videos

- VideoWriter_fourcc: https://www.programcreek.com/python/example/89348/cv2.VideoWriter_fourcc

In [30]:
# video = '/content/drive/MyDrive/vision/video/gesture1.mp4'
capture = cv2.VideoCapture(video)
connected, frame = capture.read()

In [ ]:
connected

In [ ]:
frame.shape

In [34]:
# result = '/content/drive/MyDrive/vision/Video/gesture1_result.mp4'
save_video = cv2.VideoWriter(result, cv2.VideoWriter_fourcc(*'XVID'),  10, (frame.shape[1], frame.shape[0]))

### Arms above the head in images

In [ ]:
# image2 = cv2.imread('/content/drive/MyDrive/vision/images/player.jpg')
cv2_imshow(image2)

In [ ]:
# image2 = cv2.imread('/content/drive/MyDrive/vision/images/player.jpg')
image_blob2 = cv2.dnn.blobFromImage(image = image2, scalefactor = 1.0 / 255, size = (image2.shape[1], image2.shape[0]))
network.setInput(image_blob2)
output2 = network.forward()
position_width = output2.shape[3]
position_height = output2.shape[2]
num_points = 15
points = []
threshold = 0.1
for i in range(num_points):
  confidence_map = output2[0, i, :, :]
  _, confidence, _, point = cv2.minMaxLoc(confidence_map)
  x = int((image2.shape[1] * point[0]) / position_width)
  y = int((image2.shape[0] * point[1]) / position_height)

  if confidence > threshold:
    cv2.circle(image2, (x, y), 3, (0,255,0), thickness = -1)
    cv2.putText(image2, "{}".format(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, .3, (0, 0, 255))
    cv2.putText(image2, '{}-{}'.format(point[0], point[1]), (x, y + 10), cv2.FONT_HERSHEY_SIMPLEX, .5, (255,0,255))
    points.append((x, y))
  else:
    points.append(None)

plt.figure(figsize = [14,10])
plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB));

In [38]:
def verify_arms_up(points):
  head, right_wrist, left_wrist = 0, 0, 0
  for i, point in enumerate(points):
    #print(i, point)
    if i == 0:
      head = point[1]
    elif i == 4:
      right_wrist = point[1]
    elif i == 7:
      left_wrist = point[1]

  #print(head, right_wrist, left_wrist)
  if right_wrist < head and left_wrist < head:
    return True
  else:
    return False

In [ ]:
verify_arms_up(points)